In [761]:
import numpy as np
import pandas as pd
import re

In [762]:
import networkx as nx
from collections import defaultdict

In [763]:
nodes = pd.read_csv('nodes.csv')
edges = pd.read_csv('edges.csv')
hero_net = pd.read_csv('hero-network.csv')

# Data Preprocessing
1. Some of the heroes' names in 'hero-network.csv' are not found in 'edges.csv'. This inconsistency exists for the following reasons:
- Some heroes' names in 'hero-netowrk.csv' have extra spaces at the end of their names compared to their names in 'edges.csv'.
- Some heroes' names in 'hero-netowrk.csv' have an extra '/' at the end of their names compared to their names in 'edges.csv'.
- The hero name 'SPIDER-MAN/PETER PARKER' in 'edges.csv' has been changed to 'SPIDER-MAN/PETER PAR' in 'hero-network.csv' due to a string length limit in 'hero-network.csv'.

2. Some entries in the 'hero-network.csv' have the same hero in both columns. In the graph, these entries form a self-loop. Because a self-loop makes no sense in this network, you can safely remove those from the dataset.

In [764]:
# We define a function that removes excess spaces and the character '/' at the end of the heroes' name.

def remove_extra_spaces_and_slashes(string):
    # Remove extra spaces at the end of the string
    string = re.sub(r'\s+$', '', string)

    # Remove an extra forward slash at the end of the string, if present
    string = re.sub(r'\/+$', '', string)

    return string

In [765]:
# remove extra spaces and '/' at the end of the heroes' names
# apply the function above to the two columns 
hero_net['hero1'] = hero_net['hero1'].apply(lambda row: remove_extra_spaces_and_slashes(row))

hero_net['hero2'] = hero_net['hero2'].apply(lambda row: remove_extra_spaces_and_slashes(row))

In [766]:
# We replace the wrong name with the correct name in the already cleaned columns.
hero_net['hero1'] = hero_net['hero1'].replace('SPIDER-MAN/PETER PAR', 'SPIDER-MAN/PETER PARKER')
hero_net['hero2'] = hero_net['hero2'].replace('SPIDER-MAN/PETER PAR', 'SPIDER-MAN/PETER PARKER')

In [767]:
# indices of the rows that have the same hero name in both columns
idx = hero_net[hero_net['hero1'] == hero_net['hero2']].index
len(idx)

2232

In [768]:
# before removing those rows
hero_net.shape

(574467, 2)

In [769]:
# remove those rows that have the same hero name in both columns
hero_net = hero_net.drop(idx)

In [770]:
# after removing those rows
hero_net.shape

(572235, 2)

We have removed 2232 observations.

# Graphs setup
For this homework, we are going to build two different graphs:

1. First graph: Will be constructed using the data stored in the 'hero-network.csv' file, in which an edge between two heroes can be found if they have appeared in the same comic together. The number of edges between two heroes represents the number of times they have collaborated in different comics. The graph should be considered weighted and undirected. It is up to you to decide which metric to use to calculate the weights, but we anticipate that the cost will be lower for heroes with more collaborations. Please specify which metric you used to select the weights in the report.

2. Second graph: The data in 'nodes.csv' and 'edges.csv' will be used to construct the second graph. The type of node (hero/comic) can be found in 'nodes.csv', and an edge between a hero node and a comic node can be found in 'edges.csv' when the hero has appeared in that specific comic. This graph is assumed to be undirected and unweighted.

#### 1st Graph

In [771]:
# function that creates a graph of type 1

def graph1(dataset):
    # Initialize a dictionary to store the counts of each superhero pair
    hero_pair_counts = defaultdict(int)

    # Iterate over the rows in the dataset
    for index, row in dataset.iterrows():
      # Get the values of the hero1 and hero2 columns
        hero1 = row['hero1']
        hero2 = row['hero2']

        pair = tuple(sorted([hero1, hero2]))
        hero_pair_counts[pair] += 1
    
    # initialize the graph
    G_hero_net = nx.Graph()
    
    # loop over the dataset and add the nodes & edges with the corresponding weight
    for k,v in hero_pair_counts.items():
        G_hero_net.add_edge(k[0],k[1], weight= 1/v)
    
    return G_hero_net

In [772]:
G1 = graph1(hero_net)

In [773]:
nx.info(G1)

C:\Users\yoann\AppData\Local\Temp\ipykernel_45896\968434106.py:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  nx.info(G1)


'Graph with 6421 nodes and 167100 edges'

#### 2nd graph

In [774]:
# function that creates a graph of type 2

def graph2(df_edges,df_nodes):
    # initialize the graph
    G2 = nx.MultiGraph()
    # add all nodes to the graph from the dataset
    for i in range(df_nodes.shape[0]):
        G2.add_node(df_nodes['node'].loc[i])
        G2.nodes()[df_nodes['node'].loc[i]]['type'] = df_nodes['type'].loc[i]
        
    # add all edges to the graph from the dataset
    df_edges.apply(lambda row: G2.add_edge(row['hero'], row['comic']), axis=1)
    
    return G2

In [775]:
G2 = graph2(edges,nodes)

In [776]:
nx.info(G2)

C:\Users\yoann\AppData\Local\Temp\ipykernel_45896\1976782796.py:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  nx.info(G2)


'MultiGraph with 19091 nodes and 96104 edges'

# 2. Backend Implementation
The goal of this part is the implementation of a controller system that has different functionalities. The controller should take as input an identifier "i" and run the associated function_i applied to the graph you create from the downloaded data.

Definition: As the number of nodes and edges grows, we may request to work on a subset of the data to reduce computation time and improve network visualization. In this case, we will ask you only to consider the data for top N heros. We define the top N heroes as follows:

Top N heroes: The top N heroes who have appeared in the most number of comics. The 'edges.csv' file, which represents the comics in which each hero has appeared, can be used to filter these N heroes.

Note: When the value of N is not set by the user, the function should consider the whole data.

In [777]:
def top_N_heroes(dataset, N = None):
    if N == None:
        N = len(dataset)
    
    # group the dataset by 'hero' and count the number of comics where each hero has appeared
    df_grouped = dataset.groupby('hero').agg(count = ('comic','count')).reset_index()
    df_grouped = df_grouped.sort_values(by='count', ascending=False)
    
    # take the top N heroes based on number of comics they've appeared in
    top_N = df_grouped[:N]
    
    # names of top N heroes
    names_topN = top_N['hero']
    
    return list(names_topN)

In [778]:
# # create subsets of the given datasets that have info corresponding to the top N heroes

# def create_subsets(names_heroes):
#     # create a subset of edges dataset that has info only about the top N heroes
#     edges_small = edges[edges['hero'].isin(names_heroes)]
#     edges_small = edges_small.reset_index(drop=True)

#     # hero-network.csv - Contains the network of heroes who have appeared together in the comics.
#     # We are interested in the information about the top N heroes, which means that we want all the
#     # connections/collaborations that each of those top N heroes has.
#     # Therefore, we want to keep all the rows from 'hero_network' that contain those top N heroes. 
#     # This means that, for example, hero1 might be one of the top N heroes but hero2 might not be.
#     # Thus, we will have all the information about the top N heroes from that particualr dataset.

#     # Create a subset of hero_net dataset that has info only about the top N heroes
#     hero_net_1 = hero_net[hero_net['hero1'].isin(names_heroes)]
#     hero_net_2 = hero_net[hero_net['hero2'].isin(names_heroes)]
#     hero_net_small = pd.concat([hero_net_1,hero_net_2], ignore_index=True)
#     hero_net_small = hero_net_small.reset_index(drop=True)
    
#     heroes1 = hero_net_small.hero1.unique()
#     heroes2 = hero_net_small.hero2.unique()
#     edges_heroes = edges_small.hero.unique()
#     comics = edges_small.comic.unique()
#     # names and comics from the subsets above
#     all_heroes_comics = np.concatenate([heroes1,heroes2,edges_heroes,comics])
    
#     # Create a subset of nodes dataset
#     nodes_small = nodes[nodes['node'].isin(all_heroes_comics)]
#     nodes_small = nodes_small.reset_index(drop=True)
    
#     return edges_small, nodes_small, hero_net_small

In [779]:
# edges_small, nodes_small, hero_net_small = create_subsets(names_heroes)

In [780]:
# G1_new = graph1(hero_net_small)

In [781]:
# G2_new = graph2(edges_small, nodes_small)
# nx.info(G2_new)

# Functionality 1 - extract the graph's features
Input:
- The graph data
- The graph type (ex., number 1 or number 2)
- N: denoting the top N heroes that their data should be considered

Output:
- The number of nodes in the network (if type 2, report for both node types)
- The number of collaborations of each superhero with the others (only if type 1)
- The number of heroes that have appeared in each comic (only if type 2)
- The network's density
- The network's degree distribution
- The average degree of the network
- The network's Hubs (hubs are nodes having degrees more extensive than the 95th percentile of the degree distribution)
- Whether the Network is sparse or dense


Note: For this case, it makes sense to differentiate operations between the two graphs: for example, when computing hubs for the second graph, we likely care only about comics.

In [782]:
# function that outputs the number of connections per hero
def number_connections(graph):
    # dictionary where we'll store the comics' names and the number of connections
    num_connect_per_hero = {}
    
    # loop over all nodes (heroes) and sum the reciprocal of the edge weights (weight = 1 / number of edges)
    for i in graph.nodes():
        num_connections = 0
        for k1,v1 in graph[i].items():
            for k2,v2 in v1.items():
                num_connections += 1/v2
        num_connect_per_hero[i] = int(num_connections)
    
    return num_connect_per_hero

In [798]:
# function that outputs:
# 1. number of nodes of type 'hero'
# 2. number of types of type 'comic'
# 3. number of heroes that have appeared in each comic

def number_heroes_comics(graph):
    num_heroes = 0
    num_comics = 0
    num_heroes_per_comic = {}

    for i in graph.nodes():
        if list(graph.nodes()[i].values()) == [['comic']]:
            num_comics += 1
            num_H = len(graph[i])
            num_heroes_per_comic[i] = num_H
        else:
            num_heroes += 1
    
    return num_heroes, num_comics, num_heroes_per_comic

In [799]:
# function that finds the network Hubs 
def network_hubs(graph, g_type):
    if g_type == 1:
        # dictionary key = node, value = degree of node (number of edges)
        degree_distr = dict(nx.degree(graph))
        
        # quantile 95%
        q95 = np.percentile(list(degree_distr.values()), 95)
        
        # loop over all elements of the dictionary above
        hubs = {}
        for k,v in degree_distr.items():
            # if the node's degree > 95th quantile, keep it
            if v >= q95:
                hubs[k] = v
    else:
        # take only the 'comic' nodes
        list_values = []
        for i in G2.nodes():
            if list(G2.nodes()[i].values()) == ['comic']:
                list_values.append(len(G2[i]))

        # quantile 95%
        q95 = np.percentile(list_values, 95)

        hubs = {}
        for j in G2.nodes():
            if list(G2.nodes()[j].values()) == ['comic']:
                if len(G2[j]) >= q95:
                    hubs[j] = len(G2[j])

    return hubs

In [800]:
def graph1_small(graph,names_heroes):
    # create a new graph from the original graph1 that contains only nodes for the top N heroes
    G1_small = nx.Graph()

    for i in G_hero_net.nodes():
        if i in names_heroes:
            G1_small.add_node(i)

    for edge in G_hero_net.edges():
        if (edge[0] in names_heroes) and (edge[1] in names_heroes):
            w = list(G_hero_net.edges()[edge].values())[0]
            G1_small.add_edge(edge[0],edge[1], weight = w)
    
    return G1_small

In [801]:
def graph2_small(graph,names_heroes):
    # all comics where the top N heroes appeared
    comics = set(list(edges[edges['hero'].isin(names_heroes)]['comic']))

    # create a new graph from the original graph2 that contains only nodes for the top N heroes
    G2_small = nx.Graph()

    for i in G2.nodes():
        if (i in names_heroes) or (i in comics):
            G2_small.add_node(i)
            # add the original label to the node
            G2_small.nodes()[i]['type'] = list(G2.nodes()[i].values())

    for edge in G2.edges():
        if ((edge[0] in names_heroes) and (edge[1] in comics)) or ((edge[1] in names_heroes) and (edge[0] in comics)):
            G2_small.add_edge(edge[0],edge[1])
    
    return G2_small

In [802]:
# graph type = 1 = hero-hero
# graph type = 2 = hero-comic

def Extract_G_Features(graph, g_type, N):
    if (g_type != 1) & (g_type != 2):
        return 'Wrong Graph Type'
    else:
        # extract the names of the top N heroes
        names_heroes = top_N_heroes(edges,N=N)

        if g_type == 1:
            graph_new = graph1_small(graph,names_heroes)
        else:
            graph_new = graph2_small(graph,names_heroes)

        # network's density
        density = nx.density(graph_new)

        # network's degree distribution
        degree_distr = dict(nx.degree(graph_new))

        # average degree of the network
        avg_degree = sum(degree_distr.values()) / len(graph_new.nodes)

        # network's Hubs
        hubs = network_hubs(graph_new, g_type)

        # network is dense if density > 0.5, otherwise sparse
        dense_sparse = 'dense' if density>0.5 else 'sparse'

        if g_type == 1:
            # number of nodes in the network
            number_nodes = len(graph_new.nodes())
            # number of collaborations per hero
            num_connect_per_hero = number_connections(graph_new)

            return number_nodes, num_connect_per_hero, density, degree_distr, avg_degree, hubs, dense_sparse
        else:
            # number of nodes of type 'hero' and 'comic' in the network
            # and number of heroes per comic
            num_heroes, num_comics, num_heroes_per_comic = number_heroes_comics(graph_new)

            return num_heroes, num_comics, num_heroes_per_comic, density, degree_distr, avg_degree, hubs, dense_sparse

In [804]:
names_heroes = top_N_heroes(edges,N=10)
names_heroes

['SPIDER-MAN/PETER PARKER',
 'CAPTAIN AMERICA',
 'IRON MAN/TONY STARK',
 'THING/BENJAMIN J. GR',
 'THOR/DR. DONALD BLAK',
 'HUMAN TORCH/JOHNNY S',
 'MR. FANTASTIC/REED R',
 'HULK/DR. ROBERT BRUC',
 'WOLVERINE/LOGAN',
 'INVISIBLE WOMAN/SUE']

In [796]:
Extract_G_Features(G1,g_type=1,N=10)

(10,
 {'IRON MAN/TONY STARK': 2962,
  'SPIDER-MAN/PETER PARKER': 1962,
  'HULK/DR. ROBERT BRUC': 1592,
  'CAPTAIN AMERICA': 3614,
  'WOLVERINE/LOGAN': 1040,
  'MR. FANTASTIC/REED R': 5616,
  'THING/BENJAMIN J. GR': 5688,
  'INVISIBLE WOMAN/SUE': 5324,
  'THOR/DR. DONALD BLAK': 2830,
  'HUMAN TORCH/JOHNNY S': 5708},
 1.0,
 {'IRON MAN/TONY STARK': 9,
  'SPIDER-MAN/PETER PARKER': 9,
  'HULK/DR. ROBERT BRUC': 9,
  'CAPTAIN AMERICA': 9,
  'WOLVERINE/LOGAN': 9,
  'MR. FANTASTIC/REED R': 9,
  'THING/BENJAMIN J. GR': 9,
  'INVISIBLE WOMAN/SUE': 9,
  'THOR/DR. DONALD BLAK': 9,
  'HUMAN TORCH/JOHNNY S': 9},
 9.0,
 {'IRON MAN/TONY STARK': 9,
  'SPIDER-MAN/PETER PARKER': 9,
  'HULK/DR. ROBERT BRUC': 9,
  'CAPTAIN AMERICA': 9,
  'WOLVERINE/LOGAN': 9,
  'MR. FANTASTIC/REED R': 9,
  'THING/BENJAMIN J. GR': 9,
  'INVISIBLE WOMAN/SUE': 9,
  'THOR/DR. DONALD BLAK': 9,
  'HUMAN TORCH/JOHNNY S': 9},
 'dense')

In [803]:
Extract_G_Features(G2,g_type=2,N=10)

(10,
 6049,
 {"A '00": 2,
  'A 100': 4,
  'A 101': 3,
  'A 102': 3,
  'A 103': 3,
  'A 104': 3,
  'A 105': 2,
  'A 106': 2,
  'A 107': 2,
  'A 108': 3,
  'A 109': 2,
  'A 10': 4,
  'A 110': 3,
  'A 111': 3,
  'A 112': 3,
  'A 113': 3,
  'A 114': 3,
  'A 115': 3,
  'A 116': 4,
  'A 117': 2,
  'A 118': 8,
  'A 119': 3,
  'A 11': 4,
  'A 120': 2,
  'A 121': 3,
  'A 122': 2,
  'A 123': 2,
  'A 124': 2,
  'A 125': 3,
  'A 126': 3,
  'A 127': 6,
  'A 128': 6,
  'A 129': 2,
  'A 12': 7,
  'A 130': 2,
  'A 131': 3,
  'A 132': 2,
  'A 133': 2,
  'A 134': 2,
  'A 135': 2,
  'A 137': 3,
  'A 138': 2,
  'A 139': 2,
  'A 13': 8,
  'A 140': 2,
  'A 141': 3,
  'A 142': 3,
  'A 143': 3,
  'A 144': 6,
  'A 145': 3,
  'A 146': 3,
  'A 147': 3,
  'A 148': 3,
  'A 149': 3,
  'A 14': 6,
  'A 150': 3,
  'A 151': 4,
  'A 152': 2,
  'A 153': 2,
  'A 154': 2,
  'A 155': 2,
  'A 156': 2,
  'A 157': 2,
  'A 158': 2,
  'A 159': 3,
  'A 1.5': 8,
  'A 15': 3,
  'A 160': 3,
  'A 161': 2,
  'A 162': 3,
  'A 163': 1,
